## Big Data Mining
### Project 2
### 2019-2020

Kyriakos   Zormpas  
Alexandros Zotos  
Theodora  Koutsothanasi  
Evangelia Santorinaiou  dsc19024  

## Libraries

In [23]:
#!pip install namegenerator

In [3]:
import pandas as pd
import numpy as np

import namegenerator
from datetime import datetime, timedelta
import random
import time
import names

### Dataset Creation

Our scenario in order to build our dataset is that we have all the restaurants of the world, name, id, when they were founded and their age, their coordinates, their speciality and their reviews. 

In [4]:
def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))

def random_date(start, end, prop):
    return str_time_prop(start, end, '%m/%d/%Y', prop)

To find the random latitude and longitude we used worldcitiespop.csv and we kept the country and the city that match each combination. We did not use AccentCity column because it is the same with City but with accents thus it not provides additional information. Also we removed Population because most of the values were Nan and Region because we will not use it in our scenario.

In [5]:
def get_latitude_longititude():
    df = pd.read_csv('worldcitiespop.csv')
    return df.loc[:,['Country', 'City', 'Latitude', 'Longitude']]

In [6]:
df_lat = get_latitude_longititude()[:100]
df_lat.head(5)

/home/eva/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,Country,City,Latitude,Longitude
0,ad,aixas,42.483333,1.466667
1,ad,aixirivali,42.466667,1.500000
2,ad,aixirivall,42.466667,1.500000
3,ad,aixirvall,42.466667,1.500000
4,ad,aixovall,42.466667,1.483333


### Restaurants

In [71]:
restaurant_df = pd.DataFrame(columns=['id', 'name', 'cid', 'sid', 'founded', 'age'])


for i in range(df_lat.shape[0]):
    r_date = random_date("01/01/2008", "12/31/2019", random.random())
    
    restaurant_df.loc[i] = [np.random.randint(100000, 999999), 
                                   namegenerator.gen(), 
                                   np.random.randint(10000, 99999), 
                                   np.random.randint(100000, 999999),  
                                   r_date, 
                                   int(str(datetime.today())[:4]) - int(r_date[6:])]
restaurant_df.head(5)

,id,name,cid,sid,founded,age
0,534153,trippy-sepia-fossa,40725,185724,06/10/2017,2
1,844326,gamy-cinnamon-gar,63997,243366,04/18/2013,6
2,643789,ugly-violet-rattlesnake,32194,378429,09/03/2019,0
3,343739,grumpy-lemon-rattlesnake,24482,139837,12/19/2016,3
4,324099,skinny-violet-flounder,18300,579989,12/08/2008,11


### Coordinates

In [76]:
coordinates_df = pd.DataFrame(columns = ['id', 'latitude', 'longitude', 'city', 'country_code'])

coordinates_df['id'] = restaurant_df['cid']
df_ll = get_latitude_longititude()[:100]
coordinates_df['latitude'] = df_ll['Latitude']
coordinates_df['longitude'] = df_ll['Longitude']
coordinates_df['city'] = df_ll['City']
coordinates_df['country_code'] = df_ll['Country']

coordinates_df.head(5)

,id,latitude,longitude,city,country_code
0,40725,42.483333,1.466667,aixas,ad
1,63997,42.466667,1.500000,aixirivali,ad
2,32194,42.466667,1.500000,aixirivall,ad
3,24482,42.466667,1.500000,aixirvall,ad
4,18300,42.466667,1.483333,aixovall,ad


### Speciality

In [80]:
speciality_df = pd.DataFrame(columns = [ 'id', 'name', 'calories', 'price', 'cuisine_name', 'is_vegan', 
                                        'is_gluten_free', 'is_lactose_free'])

speciality_names = [line.strip() for line in open("specialityNames.txt", 'r', encoding="utf8")]
cuisine_names = [line.strip() for line in open("cuisineNames.txt", 'r', encoding="utf8")]

for i in range(restaurant_df.shape[0]):
    speciality_df.loc[i] = [np.random.randint(10000, 99999), random.choice(speciality_names),
                            random.randrange(50, 1000), random.randrange(1, 3500), 
                            random.choice(cuisine_names), random.randrange(100) < 25
                            ,random.randrange(100) < 15, random.randrange(100) < 15]

In [81]:
speciality_df.head(5)

,id,name,calories,price,cuisine_name,is_vegan,is_gluten_free,is_lactose_free
0,88167,"Nieren Saute in Curry mit Reis, Escoffier Chutney",70,3314,German,False,False,False
1,76508,Cook & Bernheimer Co's Mount Vernon whisky,84,2942,Argentine,True,False,True
2,42607,"Reissuppe, JardiniÃ¨re",261,2998,Slovenian,False,False,False
3,94230,Brioche Cobbler with Grand Marnier Ice Cream,916,1827,Malaysian Chinese cuisine,False,False,False
4,71074,Beaujolais 1961,328,399,Hong Kong Western,False,False,False


### Reviews

In [67]:
reviews_df = pd.DataFrame(columns=['first_Name','last_Name','rating'])

for i in range(100): 
    reviews_df.loc[i] = [names.get_first_name(), names.get_last_name(), np.random.randint(1,5)]

reviews_df['id'] = restaurant_df['rid']
reviews_df.head(5)

,first_Name,last_Name,rating,id
0,Myron,Tuite,3,927108
1,Dana,Hopf,3,106415
2,Marvin,Solomon,3,763138
3,Maira,Thomas,1,759107
4,Scott,Brown,2,None


# Add Noise

* One case we thought for noise is that some restaurants either do not have specility dish or their speciality is missing. Thus in restaurant dataframe we changed some ids from speciality so that sid from restaurants and id from speciality do not much.

In [12]:
def noise_gen(df, nn, range_, col):
    
    for i in range(nn):
        df.loc[np.random.randint(0, df.shape[0]-1)][col] = np.random.randint(range_[0], range_[1])
    
    return df

In [13]:
noisy_restaurant = noise_gen(restaurant_df, int(restaurant_df.shape[0] * 1/100), (1000, 9000), 'id')
noisy_restaurant.head(5)

,id,name,cid,sid,rid,founded,age
0,626960,muggy-ultramarine-blue,293849,27299,442494,02/20/2017,2
1,522920,squirrely-crimson-falcon,430713,12538,994564,06/27/2013,6
2,275943,foggy-silver-dhole,481752,35937,532320,07/03/2011,8
3,866305,bluesy-wisteria-reindeer,571580,74719,342850,04/09/2015,4
4,328210,lumpy-asparagus-bongo,579974,22491,545374,09/06/2017,2


* Review table contains values out of range 1-5.

In [14]:
noisy_reviews = noise_gen(reviews_df, int(reviews_df.shape[0] * 1/10), (-2, 0), 'Reviews')
noisy_reviews = noise_gen(reviews_df, int(noisy_reviews.shape[0] * 1/10), (6, 10), 'Reviews')
noisy_reviews.head(5)

,First_Name,Last_Name,Reviews,id
0,Myrna,Parham,4,442494
1,Bertha,Gouty,1,994564
2,Victor,Williams,-1,532320
3,Bruce,Deans,3,342850
4,Sheila,Lokken,3,545374


* Mismatch in restaurant table between founded year and its age.

In [51]:
errors = 100
for i in range(errors):
    # throw a random number to use as index of the df
    pos = np.random.randint(0, len(restaurant_df))
    if pos%2:
        restaurant_df.loc[pos, 'age'] -= 2
    else:
        restaurant_df.loc[pos, 'age'] += 2

In [52]:
restaurant_df.head(5)

,id,name,cid,sid,rid,founded,age
0,791339,silly-khaki-lionfish,536217,45818,927108,03/07/2019,2
1,118450,sloppy-alizarin-fossa,438598,50118,106415,10/25/2012,5
2,586628,lanky-goldenrod-salamander,752458,38172,763138,03/14/2012,9
3,640398,lumpy-orchid-macaque,480159,63679,759107,11/30/2019,-2
4,155526,randy-emerald-macaw,188072,85309,437433,09/08/2015,8


* Null values insertion in random places in each dataframe

In [53]:
def add_missing_values(df, number, col=None):
    df_cols = list(df.columns)
    col_idx = col
    for i in range(number):
        pos = np.random.randint(0, len(df))
        if col == None:
            # mess up the whole dataframe randomly
            col_idx = df_cols[np.random.randint(2, len(df_cols))]

        df.loc[pos, col_idx] = None

In [65]:
null_rest  = int(100)
null_coord = int(0.1 / coordinates_df.shape[0])
null_spec  = int(0.1 / speciality_df.shape[0])
null_rev   = int(0.1 / restaurant_df.shape[0])

add_missing_values(restaurant_df, null_rest)
add_missing_values(coordinates_df,null_coord)
add_missing_values(speciality_df, null_spec)
add_missing_values(reviews_df,    null_rev)

In [66]:
restaurant_df.head(30)

,id,name,cid,sid,rid,founded,age
0,791339,silly-khaki-lionfish,536217,45818,927108,03/07/2019,2
1,118450,sloppy-alizarin-fossa,438598,None,106415,10/25/2012,5
2,586628,lanky-goldenrod-salamander,752458,38172,763138,03/14/2012,9
3,640398,lumpy-orchid-macaque,480159,63679,759107,None,-2
4,155526,randy-emerald-macaw,188072,85309,None,09/08/2015,8
5,143686,jumpy-sapphire-wasp,981358,None,182617,None,1
6,233513,breezy-seashell-audemer,170616,20952,None,09/27/2016,None
7,416928,woozy-lavender-owl,None,48573,525054,09/06/2010,7
8,462021,scanty-cornflower-nightingale,236935,79297,620676,12/16/2008,None
9,104347,sloppy-platinum-markhor,871378,69531,562157,12/18/2017,2


* Duplicate in latitude longitude  due to different names of the same city. Already existed in the file we used.